# CPSC 483-02 Project 3 Fall 2021

By Shervin Afrasiabi | shervina@csu.fullerton.edu 

Yesh Patel | yesh@csu.fullerton.edu

Vishva Patel | Vishva360@csu.fullerton.edu

### Due November 19th 2021 @ 9:45 pm

In [1]:
import pandas as pd #1
import numpy as np
import sklearn
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns', None) 

data = []
data = pd.read_csv('bank-additional-full.csv', delimiter = ';')
data.head(n = 5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


# 1

In the cell above we opened the bank additional csv and examined a few entries. 

In [2]:
from sklearn.model_selection import train_test_split #2
X = data[data.columns[0:19]]
y = data[data.columns[20]]
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.10, random_state = (2021-10-25))

# 2

We separate the training and test sets in the cell block above. 

In [3]:
X_train = X_train.drop(columns = ["duration"]) #3
X_test = X_test.drop(columns = ["duration"])


# 3

We dropped the duration columns based on the project description. 

In [4]:
X_trainCategorical = X_train[X_train.columns[0:6]]
X_testCategorical = X_test[X_test.columns[0:6]] 

X_trainCategorical = pd.get_dummies(X_trainCategorical, drop_first = True) #4
X_testCategorical = pd.get_dummies(X_testCategorical, drop_first = True)


# 4 

In the cells above we isolate the bank client data. We then encode them with pd.get_dummies().

In [5]:
from sklearn.naive_bayes import CategoricalNB #5

cnb = CategoricalNB()
cnb.fit(X_trainCategorical, y_train)
trainScore = cnb.score(X_trainCategorical, y_train)
testScore = cnb.score(X_testCategorical, y_test)
print("Train score: ",trainScore)
print("Test score: ",testScore)

Train score:  0.8818419703795625
Test score:  0.8786113134255887


# 5 

According to our results, the score is relatively accurate about 88%, but it could be better. 

In [6]:
ageList = X_train[X_train.columns[0]] #6

print("Min: ",ageList.min()," Max: ",ageList.max())

Min:  17  Max:  98


# 6


Minimum age: 17
Maximum age: 98
The age range provides 81 categories #reason

In [7]:
bins = [10,20,30,40,50,60,70,80,90] #7
TestBinnedData = X_testCategorical.copy(deep = True)
TrainedBinnedData = X_trainCategorical.copy(deep = True)

cnb_binned = CategoricalNB()

TrainedBinnedData['binned'] = pd.cut(X_train['age'],bins)
TrainedBinnedData = TrainedBinnedData.drop('age', axis = 1)
TrainedBinnedData = pd.get_dummies(TrainedBinnedData, drop_first = True) 
cnb_binned.fit(TrainedBinnedData, y_train)


TestBinnedData['binned'] = pd.cut(X_test['age'],bins)
TestBinnedData = TestBinnedData.drop('age', axis = 1)
TestBinnedData = pd.get_dummies(TestBinnedData, drop_first = True) 
cnb_binned.fit(TestBinnedData, y_test)


BinnedTrainScore = cnb_binned.score(TrainedBinnedData, y_train)
BinnedTestScore = cnb_binned.score(TestBinnedData, y_test)

print("Binned Training Score: ",BinnedTrainScore)
print("Binned Test Score: ",BinnedTestScore)

Binned Training Score:  0.8824354581995738
Binned Test Score:  0.8805535324107793


# 7 

The score's improvement was negligable, the results are almost the exact same. 

In [8]:
from sklearn.neighbors import *     #8
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_trainCategorical, y_train)
KNeighborsTrainScore = knn.score(X_trainCategorical, y_train)

knn.fit(X_testCategorical, y_test)
KNeighborsTestScore = knn.score(X_testCategorical, y_test)
print("KNeighbor Training Score: ",KNeighborsTrainScore)
print("KNeighbor Test Score: ",KNeighborsTestScore)

KNeighbor Training Score:  0.8883163829615042
KNeighbor Test Score:  0.8926924010682205


# 8

Our test score was higher compared to experiment 5, but the improvements were quite negligable(<1%).

In [9]:
yesCount = 0 #9
noCount = 0

for result in y_test: 
    if(result == "no"):
        noCount+= 1
    else:
        yesCount+= 1
print("Yes Count: ", yesCount," No Count: ", noCount)


Yes Count:  473  No Count:  3646


In [10]:
from sklearn.metrics import confusion_matrix, roc_auc_score #10
y_test.replace(('yes', 'no'),(1,0), inplace = True)
y_testzeros = np.zeros_like(y_test)
testConfusionMatrix = confusion_matrix(y_test, y_testzeros)
testRocScore = roc_auc_score(y_test, y_testzeros)
print(testConfusionMatrix, testRocScore)

[[3646    0]
 [ 473    0]] 0.5


# 9

3646 No 

473 Yes

In [11]:
#10/11 
y_testbins = cnb_binned.predict(TestBinnedData)
y_testbins = pd.DataFrame(y_testbins)
y_testbins.replace(('yes','no'),(1,0), inplace = True)

testConfusionMatrixbins = confusion_matrix(y_test, y_testbins)
testRocScorebins = roc_auc_score(y_test, y_testbins)
print("binned\n",testConfusionMatrixbins, testRocScorebins)


y_testknn = knn.predict(X_testCategorical)
y_testknn = pd.DataFrame(y_testknn)
y_testknn.replace(('yes','no'),(1,0), inplace = True)
            

testConfusionMatrixknn = confusion_matrix(y_test, y_testknn)
testRocScoreknn = roc_auc_score(y_test, y_testknn)
print("knn\n",testConfusionMatrixknn, testRocScoreknn)




binned
 [[3569   77]
 [ 415   58]] 0.5507512649618047
knn
 [[3607   39]
 [ 403   70]] 0.5686474447365644


# 10 & 11

Based on our results above, even though our score is lower, it appears that we are correctly predicting a majority of the results based on our confusion matrix. 

In [12]:
weighted = data[data.columns[20]] #12
weighted = weighted.where(weighted == 'yes')
weighted.replace(('yes'),(1), inplace = True)
data_sampled = data.sample(frac = 1, replace = True, weights = weighted, random_state = (2021-10-25))

In [13]:
data_sampled_x = data_sampled[data_sampled.columns[0:6]]
data_sampled_y = data_sampled[data_sampled.columns[20]]
data_sampled_x_categorical = pd.get_dummies(data_sampled_x, drop_first = True)
X_trainCategorical_sampled = X_trainCategorical.drop('default_yes', axis = 1)


balanced_data_x = pd.concat([X_trainCategorical_sampled, data_sampled_x_categorical])
balanced_data_y = pd.concat([y_train, data_sampled_y])

balanced_data_y.replace(('yes', 'no'),(1,0), inplace = True)

cnb_balanced = CategoricalNB()
cnb_balanced.fit(balanced_data_x, balanced_data_y)
cnb_balanced_trainScore = cnb_balanced.score(balanced_data_x, balanced_data_y)
print("Categorical Balanced Naive Bayes: ", cnb_balanced_trainScore)

bayes_bins = cnb_balanced.predict(balanced_data_x)
bayes_bins = pd.DataFrame(bayes_bins)
bayes_bins.replace(('yes','no'),(1,0), inplace = True)

bayesConfusionMatrixbins = confusion_matrix(balanced_data_y, bayes_bins)
bayesRocScorebins = roc_auc_score(balanced_data_y, bayes_bins)
print("Balanced bayes\n",bayesConfusionMatrixbins)
print("Balanced bayes auc: ", bayesRocScorebins)




Categorical Balanced Naive Bayes:  0.6241358600508581
Balanced bayes
 [[15789 17113]
 [12301 33054]]
Balanced bayes auc:  0.6043318398080197


# 12
Preprocessing

In [14]:
knn_balanced = KNeighborsClassifier(n_neighbors = 5)
knn_balanced.fit(balanced_data_x, balanced_data_y)
KNeighborsTrainScore_balanced = knn_balanced.score(balanced_data_x, balanced_data_y)
print("Knn", KNeighborsTrainScore_balanced)

balanced_knn = knn_balanced.predict(balanced_data_x)
balanced_knn = pd.DataFrame(balanced_knn)
balanced_knn.replace(('yes','no'),(1,0), inplace = True)

ConfusionMatrixknn = confusion_matrix(balanced_data_y, balanced_knn)
RocScoreknn = roc_auc_score(balanced_data_y, balanced_knn)
print("knn_balanced\n",ConfusionMatrixknn)
print("knn_balanced auc: ", RocScoreknn)



Knn 0.75997035408973
knn_balanced
 [[20325 12577]
 [ 6207 39148]]
knn_balanced auc:  0.7404449462942773


# 13

Based on the results of the experiment we can conclude that knn performs better on the balanced training set.

In [15]:
from sklearn.naive_bayes import GaussianNB

X_quan = data[data.columns[15:20]]
y_quan = data[data.columns[20]]

X_train_quan, X_test_quan, y_train_quan, y_test_quan = train_test_split(X_quan, y_quan, test_size=0.1, random_state= (2021-10-25))

X_test_quan.head()
gnb = GaussianNB()
gnb.fit(X_train_quan, y_train_quan)
trainScore_quan = gnb.score(X_train_quan, y_train_quan)
testScore_quan = gnb.score(X_test_quan, y_test_quan)
print('train: ',trainScore_quan)
print('test: ', testScore_quan)

y_train_quan.replace(('yes', 'no'),(1,0), inplace = True)
y_test_quan.replace(('yes', 'no'),(1,0), inplace = True)


y_pred_train_quan = gnb.predict(X_train_quan)
y_pred_train_quan = pd.DataFrame(y_pred_train_quan)
y_pred_train_quan.replace(('yes','no'),(1,0), inplace = True)

y_pred_test_quan = gnb.predict(X_test_quan)
y_pred_test_quan = pd.DataFrame(y_pred_test_quan)
y_pred_test_quan.replace(('yes','no'),(1,0), inplace = True)
        
ConfusionMatrix_train_quan = confusion_matrix(y_train_quan, y_pred_train_quan)
RocScore_train_quan = roc_auc_score(y_train_quan, y_pred_train_quan)
print(ConfusionMatrix_train_quan, RocScore_train_quan)
        
ConfusionMatrix_test_quan = confusion_matrix(y_test_quan, y_pred_test_quan)
RocScore_test_quan = roc_auc_score(y_test_quan, y_pred_test_quan)
print(ConfusionMatrix_test_quan, RocScore_test_quan)


train:  0.7199816558310178
test:  0.7193493566399611
[[23717  9185]
 [ 1195  2972]] 0.7170302906069623
[[2634 1012]
 [ 144  329]] 0.7089978997517046


# 14 

Based on the score, confusion matrix, and AUC score, we can say that social and economic context attributes are able to better predict the data than the previous example. 

In [18]:
data_sampled_x = data_sampled[data_sampled.columns[15:20]]
data_sampled_y = data_sampled[data_sampled.columns[20]]

balanced_data_x_quan = pd.concat([X_train_quan, data_sampled_x])
balanced_data_y_quan = pd.concat([y_train_quan, data_sampled_y])

balanced_data_y_quan.replace(('yes', 'no'),(1,0), inplace = True)



gnb_balanced = GaussianNB()
gnb_balanced.fit(balanced_data_x_quan, balanced_data_y_quan)
gnb_balanced_trainScore = gnb_balanced.score(balanced_data_x_quan, balanced_data_y_quan)
print("Gaussian Naive Bayes Score: ", gnb_balanced_trainScore)

gnb_bins = gnb_balanced.predict(balanced_data_x_quan)
gnb_bins = pd.DataFrame(gnb_bins)
gnb_bins.replace(('yes','no'),(1,0), inplace = True)

gnbConfusionMatrixbins = confusion_matrix(balanced_data_y_quan, gnb_bins)
gnbRocScorebins = roc_auc_score(balanced_data_y_quan, gnb_bins)
print("gnb Confusion Matrix: \n",gnbConfusionMatrixbins, "\ngnb ROC score: ",gnbRocScorebins)


Gaussian Naive Bayes Score:  0.7145814431935801
gnb Confusion Matrix: 
 [[23717  9185]
 [13151 32204]] 
gnb ROC score:  0.715440316603251


# 15

We were expecting the results of the experiment to change, however based on our results the balanced training set produced similar results to our previous experiment. 